# 25 Pandas Functions You Didn't Know Existed | P(Guarantee) = 0.8
## ExcelWriter, factorize, explode, squeeze, T, mask, idxmax, clip, ...
![](https://images.pexels.com/photos/5199661/pexels-photo-5199661.jpeg?auto=compress&cs=tinysrgb&dpr=2&h=650&w=940)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@introspectivedsgn?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Erik Mclean</a>
        on 
        <a href='https://www.pexels.com/photo/unrecognizable-man-in-panda-head-sitting-near-car-5199661/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels.</a> All images are by the author unless specified otherwise.
    </strong>
</figcaption>

## Setup

In [1]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

pd.options.display.max_rows = None
warnings.filterwarnings("ignore")

## Introduction

"I wish I could do this operation in Pandas…."

Well, chances are, you can!

Pandas is so vast and deep that it enables you to execute virtually any tabular manipulation you can think of. However, this vastness sometimes comes at a disadvantage.

Many elegant features that solve rare edge-cases, unique scenarios are lost in the documentation, shadowed by the more frequently used functions.

This article aims to rediscover those features and show you that Pandas is more capable than you ever knew.

## 1. `ExcelWriter`

`ExcelWriter` is a generic class for creating excel files (with sheets!) and writing DataFrames to them. Let's say we have these 2:

In [2]:
# Load two datasets
diamonds = sns.load_dataset("diamonds")
tips = sns.load_dataset("tips")

# Write to the same excel file
with pd.ExcelWriter("data/data.xlsx") as writer:

    diamonds.to_excel(writer, sheet_name="diamonds")
    tips.to_excel(writer, sheet_name="tips")

It has additional attributes to specify the DateTime format to be used, whether you want to create a new excel file or modify an existing one, what happens when a sheet exists, etc. Check out the details from the [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.ExcelWriter.html).

## 2. `pipe`

![](https://images.pexels.com/photos/192473/pexels-photo-192473.jpeg?auto=compress&cs=tinysrgb&dpr=2&h=650&w=940)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@leah-kelley-50725?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Leah Kelley</a>
        on 
        <a href='https://www.pexels.com/photo/grayscale-photo-of-man-holding-tobacco-pipe-192473/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels</a>
    </strong>
</figcaption>

`pipe` is one of the best functions for doing data cleaning in a concise, compact manner in Pandas. It allows you to chain multiple custom functions into a single operation.

For example, let's say you have functions to `drop_duplicates`, `remove_outliers`, `encode_categoricals` that accept their own arguments. Here is how you apply all three in a single operation:

```python
df_preprocessed = (diamonds.pipe(drop_duplicates).
                            pipe(remove_outliers, ['price', 'carat', 'depth']).
                            pipe(encode_categoricals, ['cut', 'color', 'clarity'])
                  )
```

I like how this function resembles [Sklearn pipelines](https://towardsdatascience.com/how-to-use-sklearn-pipelines-for-ridiculously-neat-code-a61ab66ca90d). There is more you can do with it, so check out the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pipe.html) or this [helpful article](https://towardsdatascience.com/a-better-way-for-data-preprocessing-pandas-pipe-a08336a012bc).

## 3. `factorize`

This function is a pandas alternative to Sklearn's `LabelEncoder`:

In [3]:
# Mind the [0] at the end
diamonds["cut_enc"] = pd.factorize(diamonds["cut"])[0]

diamonds["cut_enc"].sample(5)

52103    2
39813    0
31843    0
10675    0
6634     0
Name: cut_enc, dtype: int64

Unlike `LabelEncoder`, `factorize` returns a tuple of two values: the encoded column and a list of the unique categories:

In [4]:
codes, unique = pd.factorize(diamonds["cut"], sort=True)

codes[:10]

array([0, 1, 3, 1, 3, 2, 2, 2, 4, 2], dtype=int64)

In [5]:
unique

CategoricalIndex(['Ideal', 'Premium', 'Very Good', 'Good', 'Fair'], categories=['Ideal', 'Premium', 'Very Good', 'Good', 'Fair'], ordered=False, dtype='category')

## 4. `explode` - 🤯🤯

![](https://images.unsplash.com/photo-1567446042109-8a62d37fea07?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=750&q=80)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://unsplash.com/@joshuas?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText'>Joshua Sukoff</a>
        on 
        <a href='https://unsplash.com/s/photos/explode?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText'>Unsplash</a>
    </strong>
</figcaption>

A function with an interesting name is `explode`. Let's see an example first and then, explain:

In [6]:
data = pd.Series([1, 6, 7, [46, 56, 49], 45, [15, 10, 12]]).to_frame("dirty")
data

,dirty
0,1
1,6
2,7
3,"[46, 56, 49]"
4,45
5,"[15, 10, 12]"


The `dirty` column has two rows where values are recorded as actual lists. You may often see this type of data in surveys as some questions accept multiple answers.

In [7]:
data.explode("dirty", ignore_index=True)

,dirty
0,1
1,6
2,7
3,46
4,56
5,49
6,45
7,15
8,10
9,12


`explode` takes a cell with an array-like value and explodes it into multiple rows. Set `ignore_index` to True to keep the ordering of a numeric index.

## 5. `squeeze`

![](https://images.pexels.com/photos/5875701/pexels-photo-5875701.jpeg?auto=compress&cs=tinysrgb&dpr=2&h=650&w=940)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@cottonbro?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>cottonbro</a>
        on 
        <a href='https://www.pexels.com/photo/close-up-photo-of-sausage-5875701/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels.</a>
    </strong>
</figcaption>

Another function with a funky name is `squeeze` and is used in very rare but annoying edge cases.

One of these cases is when a single value is returned from a condition used to subset a DataFrame. Consider this example:

In [8]:
subset = diamonds.loc[diamonds.index < 1, ["price"]]
subset

,price
0,326


Even though there is just one cell, it is returned as a DataFrame. This can be annoying since you now have to use `.loc` again with both the column name and index to access the price.

But, if you know `squeeze`, you don't have to. The function enables you to remove an axis from a single-cell DataFrame or Series. For example:

In [9]:
subset.squeeze()

326

Now, only the scalar is returned. It is also possible to specify the axis to remove:

In [10]:
subset.squeeze("columns")  # or "rows"

0    326
Name: price, dtype: int64

Note that `squeeze` only works for DataFrames or Series with single values.

## 6. between

![](https://images.unsplash.com/photo-1596806082495-fb05004cfe49?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=769&q=80)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://unsplash.com/@jujudreaminx?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText'>Justin Dream</a>
        on 
        <a href='https://unsplash.com/s/photos/between?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText'>Pexels</a>
    </strong>
</figcaption>

A rather nifty function for boolean indexing numeric features within a range:

In [11]:
# Get diamonds that are priced between 3500 and 3700 dollars
diamonds[diamonds["price"].between(3500, 3700, inclusive="neither")].sample(5)

,carat,cut,color,clarity,depth,table,price,x,y,z,cut_enc
4358,1.10,Premium,I,SI2,62.9,59.0,3604,6.60,6.53,4.13,1
4639,0.73,Ideal,G,VVS1,61.9,57.0,3669,5.76,5.80,3.58,0
4024,0.73,Ideal,F,VVS2,61.7,54.0,3524,5.76,5.82,3.57,0
4227,1.20,Ideal,I,I1,61.9,56.0,3570,6.85,6.82,4.23,0
3987,0.82,Premium,F,VS1,62.1,56.0,3513,6.02,5.97,3.72,1


## 7. `T`

![](https://images.pexels.com/photos/258083/pexels-photo-258083.jpeg?auto=compress&cs=tinysrgb&dpr=2&h=650&w=940)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@pixabay?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pixabay</a>
        on 
        <a href='https://www.pexels.com/photo/close-up-of-illuminated-text-against-black-background-258083/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels</a>
    </strong>
</figcaption>

All DataFrames have a simple `T` attribute, which stands for transpose. You may not use it often, but I find it quite useful when displaying DataFrames of the `describe` method:

In [12]:
## HIDE
from sklearn.datasets import load_boston

bunch = load_boston()
boston = pd.DataFrame(bunch["data"], columns=bunch["feature_names"])

In [13]:
boston.describe().T.head(10)

,count,mean,std,min,25%,50%,75%,max
CRIM,506.0,3.613524,8.601545,0.00632,0.082045,0.25651,3.677083,88.9762
ZN,506.0,11.363636,23.322453,0.00000,0.000000,0.00000,12.500000,100.0000
INDUS,506.0,11.136779,6.860353,0.46000,5.190000,9.69000,18.100000,27.7400
CHAS,506.0,0.069170,0.253994,0.00000,0.000000,0.00000,0.000000,1.0000
NOX,506.0,0.554695,0.115878,0.38500,0.449000,0.53800,0.624000,0.8710
RM,506.0,6.284634,0.702617,3.56100,5.885500,6.20850,6.623500,8.7800
AGE,506.0,68.574901,28.148861,2.90000,45.025000,77.50000,94.075000,100.0000
DIS,506.0,3.795043,2.105710,1.12960,2.100175,3.20745,5.188425,12.1265
RAD,506.0,9.549407,8.707259,1.00000,4.000000,5.00000,24.000000,24.0000
TAX,506.0,408.237154,168.537116,187.00000,279.000000,330.00000,666.000000,711.0000


The Boston housing dataset has 30 numeric columns. If you call `describe` as-is, the DataFrame will stretch horizontally, making it hard to compare the statistics. Taking the transpose will switch the axes so that summary statistics are given in columns.

## 8. Pandas Styler

Did you know that Pandas allows you to style DataFrames?

They have a `style` attribute which opens doors to customizations and styles only limited by your HTML and CSS knowledge. I won't discuss the full details of what you can do with `style` but only show you my favorite functions:

In [14]:
## HIDE
diabetes = pd.read_csv("data/diabetes.csv")

In [15]:
diabetes.describe().T.drop("count", axis=1).style.highlight_max(color="darkred")

,mean,std,min,25%,50%,75%,max
Pregnancies,3.845052,3.369578,0.000000,1.000000,3.000000,6.000000,17.000000
Glucose,120.894531,31.972618,0.000000,99.000000,117.000000,140.250000,199.000000
BloodPressure,69.105469,19.355807,0.000000,62.000000,72.000000,80.000000,122.000000
SkinThickness,20.536458,15.952218,0.000000,0.000000,23.000000,32.000000,99.000000
Insulin,79.799479,115.244002,0.000000,0.000000,30.500000,127.250000,846.000000
BMI,31.992578,7.884160,0.000000,27.300000,32.000000,36.600000,67.100000
DiabetesPedigreeFunction,0.471876,0.331329,0.078000,0.243750,0.372500,0.626250,2.420000
Age,33.240885,11.760232,21.000000,24.000000,29.000000,41.000000,81.000000
Outcome,0.348958,0.476951,0.000000,0.000000,0.000000,1.000000,1.000000


Above, we are highlighting cells that hold the maximum value of a column. Another cool styler is `background_gradient` which can give columns a gradient background color based on their values:

In [16]:
diabetes.describe().T.drop("count", axis=1).style.background_gradient(
    subset=["mean", "50%"], cmap="Reds"
)

,mean,std,min,25%,50%,75%,max
Pregnancies,3.845052,3.369578,0.000000,1.000000,3.000000,6.000000,17.000000
Glucose,120.894531,31.972618,0.000000,99.000000,117.000000,140.250000,199.000000
BloodPressure,69.105469,19.355807,0.000000,62.000000,72.000000,80.000000,122.000000
SkinThickness,20.536458,15.952218,0.000000,0.000000,23.000000,32.000000,99.000000
Insulin,79.799479,115.244002,0.000000,0.000000,30.500000,127.250000,846.000000
BMI,31.992578,7.884160,0.000000,27.300000,32.000000,36.600000,67.100000
DiabetesPedigreeFunction,0.471876,0.331329,0.078000,0.243750,0.372500,0.626250,2.420000
Age,33.240885,11.760232,21.000000,24.000000,29.000000,41.000000,81.000000
Outcome,0.348958,0.476951,0.000000,0.000000,0.000000,1.000000,1.000000


This feature comes especially handy when you are using `describe` on a table with many columns and want to compare summary statistics. Check out the documentation of the styler [here](https://pandas.pydata.org/docs/reference/style.html).

## 9. Pandas options

Like Matplotlib, pandas has global settings that you can tweak to change the default behaviors:

In [17]:
dir(pd.options)

['compute', 'display', 'io', 'mode', 'plotting']

These settings are divided into 5 modules. Let's see what settings are there under `display`:

In [18]:
dir(pd.options.display)

['chop_threshold',
 'colheader_justify',
 'column_space',
 'date_dayfirst',
 'date_yearfirst',
 'encoding',
 'expand_frame_repr',
 'float_format',
 'html',
 'large_repr',
 'latex',
 'max_categories',
 'max_columns',
 'max_colwidth',
 'max_info_columns',
 'max_info_rows',
 'max_rows',
 'max_seq_items',
 'memory_usage',
 'min_rows',
 'multi_sparse',
 'notebook_repr_html',
 'pprint_nest_depth',
 'precision',
 'show_dimensions',
 'unicode',
 'width']

There are options under `display` but I mostly use `max_columns` and `precision`:

In [19]:
# Remove the limit to display the number of cols
pd.options.display.max_columns = None

# Only show 5 numbers after the decimal
pd.options.display.precision = 5  # gets rid of scientific notation

You can check out the [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html) to dig deeper into this wonderful feature.

## 10. `convert_dtypes`

We all know that pandas has an annoying tendency to mark some columns as `object` data type. Instead of manually specifying their types, you can use `convert_dtypes` method which tries to infer the best data type:

In [20]:
sample = pd.read_csv(
    "data/station_day.csv",
    usecols=["StationId", "CO", "O3", "AQI_Bucket"],
)
sample.dtypes

StationId      object
CO            float64
O3            float64
AQI_Bucket     object
dtype: object

In [21]:
sample.convert_dtypes().dtypes

StationId      string
CO            float64
O3            float64
AQI_Bucket     string
dtype: object

Unfortunately, it can't pares dates due to the caveats of different date time formats.

## 11. `select_dtypes`

A function I use all the time is `select_dtypes`. I think it is obvious what the function does from its name. It has `include` and `exclude` parameters that you can use to select columns including or excluding certain data types.

For example, choose only numeric columns with `np.number`:

In [22]:
# Choose only numerical columns
diamonds.select_dtypes(include=np.number).head()

,carat,depth,table,price,x,y,z,cut_enc
0,0.23,61.5,55.0,326,3.95,3.98,2.43,0
1,0.21,59.8,61.0,326,3.89,3.84,2.31,1
2,0.23,56.9,65.0,327,4.05,4.07,2.31,2
3,0.29,62.4,58.0,334,4.20,4.23,2.63,1
4,0.31,63.3,58.0,335,4.34,4.35,2.75,2


Or `exclude` them:

In [23]:
# Exclude numerical columns
diamonds.select_dtypes(exclude=np.number).head()

,cut,color,clarity
0,Ideal,E,SI2
1,Premium,E,SI1
2,Good,E,VS1
3,Premium,I,VS2
4,Good,J,SI2


## 12. `mask`

![](https://images.pexels.com/photos/38275/anonymous-studio-figure-photography-facial-mask-38275.jpeg?auto=compress&cs=tinysrgb&dpr=2&h=650&w=940)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@pixabay?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pixabay</a>
        on 
        <a href='https://www.pexels.com/photo/photo-of-guy-fawkes-mask-with-red-flower-on-top-on-hand-38275/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels.</a>
    </strong>
</figcaption>

`mask` allows you to quickly replace cell values where a custom condition is true. 

For example, let's say we have a survey data collected from people aged 50-60.

In [24]:
# Create sample data
ages = pd.Series([55, 52, 50, 66, 57, 59, 49, 60]).to_frame("ages")

ages

,ages
0,55
1,52
2,50
3,66
4,57
5,59
6,49
7,60


We will treat ages that are outside 50-60 range (there are two, 49 and 66) as data entry mistakes and replace them with NaNs.

In [25]:
ages.mask(cond=~ages["ages"].between(50, 60), other=np.nan)

,ages
0,55.0
1,52.0
2,50.0
3,NaN
4,57.0
5,59.0
6,NaN
7,60.0


So, `mask` replaces values that don't meet `cond` with `other`.

## 13. `min` and `max` along the columns axis

Even though `min` and `max` functions are well-known, they have another useful property for some edge-cases. Consider this dataset:

In [26]:
index = ["Diamonds", "Titanic", "Iris", "Heart Disease", "Loan Default"]
libraries = ["XGBoost", "CatBoost", "LightGBM", "Sklearn GB"]
df = pd.DataFrame(
    {lib: np.random.uniform(90, 100, 5) for lib in libraries}, index=index
)

df

,XGBoost,CatBoost,LightGBM,Sklearn GB
Diamonds,99.42441,99.27410,99.52684,95.72998
Titanic,90.86399,94.25364,92.10503,99.63650
Iris,99.10989,98.15600,96.34995,92.47787
Heart Disease,97.21279,95.02175,99.31627,97.29571
Loan Default,97.96728,90.63744,96.46238,90.89748


The above fake DataFrame is a point-performance of 4 different gradient boosting libraries on 5 datasets. We want to find the library that performed best at each dataset. Here is how you do it elegantly with `max`:

In [27]:
df.max(axis=1)

Diamonds         99.52684
Titanic          99.63650
Iris             99.10989
Heart Disease    99.31627
Loan Default     97.96728
dtype: float64

Just change the axis to 1 and you get a row-wise max/min. 

## 14. `nlargest` and `nsmallest`

Sometimes you don't just want the min/max of a column. You want to see the top N or ~(top N) values of a variable. This is where `nlargest` and `nsmallest` comes in handy.

Let's see the top 5 most expensive and cheapest diamonds:

In [28]:
diamonds.nlargest(5, "price")

,carat,cut,color,clarity,depth,table,price,x,y,z,cut_enc
27749,2.29,Premium,I,VS2,60.8,60.0,18823,8.50,8.47,5.16,1
27748,2.00,Very Good,G,SI1,63.5,56.0,18818,7.90,7.97,5.04,3
27747,1.51,Ideal,G,IF,61.7,55.0,18806,7.37,7.41,4.56,0
27746,2.07,Ideal,G,SI2,62.5,55.0,18804,8.20,8.13,5.11,0
27745,2.00,Very Good,H,SI1,62.8,57.0,18803,7.95,8.00,5.01,3


In [29]:
diamonds.nsmallest(5, "price")

,carat,cut,color,clarity,depth,table,price,x,y,z,cut_enc
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43,0
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31,1
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31,2
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63,1
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75,2


## 15. `idxmax` and `idxmin`

When you call `max` or `min` on a column, pandas returns the value that is largest/smallest. However, sometimes you want the *position* of the min/max, which is not possible with these functions.

Instead, you should use `idxmax`/`idxmin`:

In [30]:
diamonds.price.idxmax()

27749

In [31]:
diamonds.carat.idxmin()

14

You can also specify the `columns` axis, in which case the functions return the index number of the column.

## 16. `value_counts` with `dropna=False`

A common operation to find the percentage of missing values in a column is to chain `isnull` and `sum` and divide by the length of the array. 

But, you can do the same thing with `value_counts` with relevant arguments:

In [32]:
ames_housing = pd.read_csv("data/train.csv")

ames_housing["FireplaceQu"].value_counts(dropna=False, normalize=True)

NaN    0.47260
Gd     0.26027
TA     0.21438
Fa     0.02260
Ex     0.01644
Po     0.01370
Name: FireplaceQu, dtype: float64

Fireplace quality of Ames housing dataset consists of 47% nulls.

## 17. `clip`

![](https://images.pexels.com/photos/2448452/pexels-photo-2448452.jpeg?auto=compress&cs=tinysrgb&dpr=2&h=650&w=940)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@ann-h-45017?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Ann H</a>
        on 
        <a href='https://www.pexels.com/photo/a-lot-of-paper-clips-2448452/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels</a></strong>
</figcaption>

Outlier detection and removal is common in data analysis. 

`clip` function makes it really easy to find outliers outside a range and replacing them with the hard limits. 

Let's go back to the ages example:

In [33]:
ages

,ages
0,55
1,52
2,50
3,66
4,57
5,59
6,49
7,60


This time, we will replace the out-of-range ages with the hard limits of 50 and 60:

In [34]:
ages.clip(50, 60)

,ages
0,55
1,52
2,50
3,60
4,57
5,59
6,50
7,60


Fast and efficient!

## 18. `at_time` and `between_time`

These two can be useful when working with time-series that have high granularity. 

`at_time` allows you to subset values at a specific date or time. Consider this time series:

In [35]:
index = pd.date_range("2021-08-01", periods=100, freq="H")
data = pd.DataFrame({"col": list(range(100))}, index=index)

data.head()

,col
2021-08-01 00:00:00,0
2021-08-01 01:00:00,1
2021-08-01 02:00:00,2
2021-08-01 03:00:00,3
2021-08-01 04:00:00,4


Let's select all rows at 3 PM:

In [36]:
data.at_time("15:00")

,col
2021-08-01 15:00:00,15
2021-08-02 15:00:00,39
2021-08-03 15:00:00,63
2021-08-04 15:00:00,87


Cool, huh? Now, let's use `between_time` to select rows within a custom interval:

In [37]:
from datetime import datetime

data.between_time("09:45", "12:00")

,col
2021-08-01 10:00:00,10
2021-08-01 11:00:00,11
2021-08-01 12:00:00,12
2021-08-02 10:00:00,34
2021-08-02 11:00:00,35
2021-08-02 12:00:00,36
2021-08-03 10:00:00,58
2021-08-03 11:00:00,59
2021-08-03 12:00:00,60
2021-08-04 10:00:00,82


Note that both functions require a DateTimeIndex and they only work with times (as in *o'clock*). If you want to subset within a DateTime interval, use `between`.

## 19. `bdate_range`

`bdate_range` is a short-hand function to create TimeSeries indices with business-day frequency:

In [38]:
series = pd.bdate_range("2021-01-01", "2021-01-31")  # A period of one month
len(series)

21

Business-day frequencies are common in the financial world. So, this function may come in handy when reindexing existing time-series with `reindex` function.

## 20. `autocorr`

One of the critical components in time-series analysis is examining the autocorrelation of a variable. 

Autocorrelation is the plain-old correlation coefficient but it is calculated with the lagging version of a time series. 

In more detail, autocorrelation of a time series at `lag=k` is calculated as follows:

1. The time-series is shifted till `k` periods:

In [39]:
## HIDE
# Prep the data for an example
dt = pd.date_range("2021-01-01", periods=len(tips))
tips.index = dt

time_series = tips[["tip"]]

In [40]:
time_series["lag_1"] = time_series["tip"].shift(1)
time_series["lag_2"] = time_series["tip"].shift(2)
time_series["lag_3"] = time_series["tip"].shift(3)
time_series["lag_4"] = time_series["tip"].shift(4)
# time_series['lag_k'] = time_series['tip'].shift(k)

time_series.head()

,tip,lag_1,lag_2,lag_3,lag_4
2021-01-01,1.01,NaN,NaN,NaN,NaN
2021-01-02,1.66,1.01,NaN,NaN,NaN
2021-01-03,3.50,1.66,1.01,NaN,NaN
2021-01-04,3.31,3.50,1.66,1.01,NaN
2021-01-05,3.61,3.31,3.50,1.66,1.01


2. Correlation is calculated between the original `tip` and each `lag_*`. 

Instead of doing all this manually, you can use the `autocorr` function of Pandas:

In [41]:
# Autocorrelation of tip at lag_10
time_series["tip"].autocorr(lag=8)

0.07475238789967077

You can read more about the importance of autocorrelation in time-series analysis from this [post](https://towardsdatascience.com/advanced-time-series-analysis-in-python-decomposition-autocorrelation-115aa64f475e).

## 21. `hasnans`

Pandas offers a quick method to check if a given series contains any nulls with `hasnans` attribute:

In [42]:
series = pd.Series([2, 4, 6, "sadf", np.nan])
series.hasnans

True

According to its [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.hasnans.html), it enables various performance increases. Note that the attribute works only on `pd.Series`.

## 22. `at` and `iat`

These two accessors are much faster alternatives to `loc` and `iloc` with a disadvantage. They only allow selecting or replacing a single value at a time:

In [43]:
# [index, label]
diamonds.at[234, "cut"]

'Ideal'

In [44]:
# [index, index]
diamonds.iat[1564, 4]

61.2

In [45]:
# Replace 16541th row of the price column
diamonds.at[16541, "price"] = 10000

## 23. `argsort`

You should use this function when you want to extract the indices that would sort an array:

In [46]:
tips.reset_index(inplace=True, drop=True)

sort_idx = tips["total_bill"].argsort(kind="mergesort")

# Now, sort `tips` based on total_bill
tips.iloc[sort_idx].head()

,total_bill,tip,sex,smoker,day,time,size
67,3.07,1.00,Female,Yes,Sat,Dinner,1
92,5.75,1.00,Female,Yes,Fri,Dinner,2
111,7.25,1.00,Female,No,Sat,Dinner,1
172,7.25,5.15,Male,Yes,Sun,Dinner,2
149,7.51,2.00,Male,No,Thur,Lunch,2


## 24. `cat` accessor

It is common knowledge that Pandas enables to use built-in Python functions on dates and strings using accessors like `dt` or `str`. 

Pandas also has a special `category` data type for categorical variables as can be seen below:

In [47]:
diamonds.dtypes

carat       float64
cut        category
color      category
clarity    category
depth       float64
table       float64
price         int64
x           float64
y           float64
z           float64
cut_enc       int64
dtype: object

When a column is `category`, you can use several special functions using the `cat` accessor. For example, let's see the unique categories of diamond cuts:

In [48]:
diamonds["cut"].cat.categories

Index(['Ideal', 'Premium', 'Very Good', 'Good', 'Fair'], dtype='object')

There are also functions like `remove_categories` or `rename_categories`, etc.:

In [49]:
diamonds["new_cuts"] = diamonds["cut"].cat.rename_categories(list("ABCDE"))
diamonds["new_cuts"].cat.categories

Index(['A', 'B', 'C', 'D', 'E'], dtype='object')

You can see the full list of functions under the `cat` accessor [here](https://pandas.pydata.org/pandas-docs/stable/reference/series.html#categorical-accessor).

## 25. `GroupBy.nth`

This function only works with `GroupBy` objects. Specifically, after grouping, `nth` returns the nth row from each group:

In [50]:
diamonds.groupby("cut").nth(5)

,carat,color,clarity,depth,table,price,x,y,z,cut_enc,new_cuts
cut,,,,,,,,,,,
Ideal,0.33,I,SI2,61.2,56.0,403,4.49,4.50,2.75,0,A
Premium,0.24,I,VS1,62.5,57.0,355,3.97,3.94,2.47,1,B
Very Good,0.23,E,VS2,63.8,55.0,352,3.85,3.92,2.48,3,C
Good,0.30,I,SI2,63.3,56.0,351,4.26,4.30,2.71,2,D
Fair,0.91,H,SI2,64.4,57.0,2763,6.11,6.09,3.93,4,E


## Summary

Even though libraries like Dask and datatable are slowly winning over Pandas with their shiny new features for handling massive datasets, Pandas still remains the most widely-used data manipulation tool in Python data science ecosystem.

The library still remains as a role-model for other packages to imitate and improve upon, as it integrates into the modern SciPy stack so well. Thank you for reading!